In [1]:
# !mkdir wids_data_thon 
# !tar -xf WiDSWorldWide_GlobalDathon26.zip -C wids_data_thon

Overview

    When a wildfire ignites, emergency managers face an impossible question with incomplete information. Which fires will reach populated areas? How quickly? And which communities should prepare first?
    
    This year's WiDS Global Datathon challenges participants to build survival models that answer these questions using only the earliest signals available. Your task is to predict the probability that a wildfire will threaten an evacuation zone within 12, 24, 48, and 72 hours, drawing on data from just the first five hours after ignition.
    
    The goal is not a single prediction but a calibrated forecast across multiple time horizons. Emergency responders need both urgency rankings (which fires demand immediate attention) and probability estimates they can trust when making high-stakes decisions about evacuations, resource deployment, and public alerts.

I. Non-Parametric Models (Descriptive)

These models are primarily used for visualizing and describing the data without assuming a specific underlying distribution.


Algorithm,What it Does,Key Specifications/Assumptions:

        Kaplan-Meier (KM) Estimator,"Estimates the survival function S(t) from lifetime data. It is the ""gold standard"" for visualizing survival curves.",Non-parametric. Does not require a specific distribution. Assumes censoring is independent of survival.
        Nelson-Aalen Estimator,Estimates the cumulative hazard function H(t). Useful for understanding the risk over time.,Non-parametric. Robust for small samples. Good for identifying the rate of event occurrences.
        Log-Rank Test,A statistical test used to compare the survival distributions of two or more independent groups.,Non-parametric. Tests the null      hypothesis that there is no difference between the populations.


II. Semi-Parametric Models (Regression)

These models allow you to assess the effect of multiple variables (covariates) on survival time.


Algorithm,What it Does,Key Specifications/Assumptions

    Cox Proportional Hazards (CPH),Models the relationship between covariates and the hazard rate. It is the most widely used regression model.,"Proportional Hazards Assumption: Assumes the ratio of hazards for any two individuals is constant over time. It is semi-parametric because the ""baseline hazard"" is unspecified."
    Regularized Cox (Lasso/Ridge),An extension of CPH that adds penalties (L1/L2) to the likelihood function to handle high-dimensional data.,Used when you have many features (more than observations). Prevents overfitting and performs feature selection.



III. Parametric Models

These models assume the survival times follow a specific mathematical distribution.


Algorithm,What it Does,Key Specifications/Assumptions

    Weibull Model,"A versatile model where the hazard rate can increase, decrease, or remain constant over time.",Parametric. Requires the data to fit a Weibull distribution. Very common in engineering and reliability.
    Exponential Model,A simpler version of Weibull where the hazard rate is strictly constant.,"Parametric. Assumes ""memoryless"" property—the risk of the event doesn't change regardless of how long the subject has survived."
    Accelerated Failure Time (AFT),"Instead of modeling the hazard, it models the log of the survival time as a linear function of covariates.",Parametric. Assumes that the effect of a covariate is to accelerate or decelerate the life course of a subject.

IV. Machine Learning & Deep Learning Models

Modern approaches designed to capture non-linear relationships and complex interactions.


Algorithm,What it Does,Key Specifications/Assumptions

    Random Survival Forest (RSF),An ensemble of survival trees. It averages the results of many trees to predict survival.,Non-linear. Does not require the Proportional Hazards assumption. Excellent for handling high-dimensional data and complex interactions.
    DeepSurv,A deep learning approach that uses a Neural Network to predict the hazard ratio in a CPH-like framework.,Deep Learning. Effectively models non-linear effects of covariates. Requires larger datasets compared to traditional CPH.
    MTLR (Multi-Task Logistic Regression),Models the survival probability across multiple discrete time intervals simultaneously.,Multi-task. Estimates the entire survival distribution rather than just a single risk score. Good for personalized survival curves.



Which model should you choose?

    For simple visualization: Use Kaplan-Meier.

    For standard medical/clinical research: Use Cox Proportional Hazards.

    For complex, high-dimensional data (e.g., genomics or tech logs): Use Random Survival Forest or DeepSurv.

    For physical engineering/failure analysis: Use Weibull.

Would you like me to provide a Python code template using the lifelines or pysurvival library to implement one of these?

In [2]:
# from sksurv.tree import SurvivalTree

In [3]:
# importing libraries
import math
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

%matplotlib inline

from lifelines.datasets import load_dd
from lifelines import KaplanMeierFitter


train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
sub=pd.read_csv("sample_submission.csv")
train.head()

,event_id,num_perimeters_0_5h,dt_first_last_0_5h,low_temporal_resolution_0_5h,area_first_ha,area_growth_abs_0_5h,area_growth_rel_0_5h,area_growth_rate_ha_per_h,log1p_area_first,log1p_growth,...,dist_fit_r2_0_5h,alignment_cos,alignment_abs,cross_track_component,along_track_speed,event_start_hour,event_start_dayofweek,event_start_month,time_to_hit_hours,event
0,10892457,3,4.265188,0,79.696304,2.875935,0.036086,0.674281,4.390693,1.354787,...,0.886373,-0.054649,0.054649,-1.937219,-0.106026,19,4,5,18.892512,0
1,11757157,2,1.169918,0,8.946749,0.000000,0.000000,0.000000,2.297246,0.000000,...,0.000000,-0.568898,0.568898,-0.000000,-0.000000,4,4,6,22.048108,1
2,11945086,4,4.777526,0,106.482638,0.000000,0.000000,0.000000,4.677329,0.000000,...,0.000000,0.882385,0.882385,0.000000,0.000000,22,4,8,0.888895,1
3,12044083,1,0.000000,1,67.631125,0.000000,0.000000,0.000000,4.228746,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20,5,8,60.953021,0
4,12052347,2,4.975273,0,35.632874,0.000000,0.000000,0.000000,3.600946,0.000000,...,0.000000,0.934634,0.934634,-0.000000,0.000000,21,5,7,44.990274,0


In [4]:
train.event.value_counts()

event
0    152
1     69
Name: count, dtype: int64

In [5]:
# train.describe()

In [6]:
from sksurv.util import Surv

# From two separate arrays or lists
# y_train = Surv.from_arrays(event=train['event'], time=train['time_to_hit_hours'])

# OR directly from a DataFrame
# y_train = Surv.from_dataframe(event='event', time='time_to_hit_hours', data=train)

# Do the same for your test data
# y_test = Surv.from_arrays(event=test['event'], time=test['time_to_hit_hours'])

X = train.iloc[:,1:-2] 
X_test = test.iloc[:,1:] 

In [7]:
from sksurv.ensemble import RandomSurvivalForest

# rsf = RandomSurvivalForest(n_estimators=100)
# rsf.fit(X_train, y_train)

# Get the survival curves
# surv_funcs = rsf.predict_survival_function(X_test)

# Calculate the area under the curve (Mean Survival Time)
# import numpy as np
# predicted_hours = [np.trapz(s.y, s.x) for s in surv_funcs]

# X_test['predicted_hours'] = predicted_hours

# import numpy as np

# # Define the competition time points
# time_points = [12, 24, 48, 72]

# # Create a helper function to safely extract the probability
# Single safe probability for one survival function
def get_safe_prob(s_func, t, eps=1e-6):
    """
    Return event probability 1 - S(t) safely,
    clipping t to the survival function's domain.
    """
    max_t = s_func.domain[1]
    safe_t = min(max(t, eps), max_t)
    return 1.0 - s_func(safe_t)



def extract_event_probs(surv_funcs, time_points, eps=1e-6):
    """
    Convert survival functions to event probabilities at given time points.
    surv_funcs: list of StepFunction objects
    time_points: list of time horizons
    """
    probs = np.zeros((len(surv_funcs), len(time_points)))
    for i, fn in enumerate(surv_funcs):
        max_t = fn.domain[1]
        for j, t in enumerate(time_points):
            safe_t = min(max(t, eps), max_t)
            probs[i, j] = 1.0 - fn(safe_t)
    return probs



In [8]:
from sksurv.metrics import concordance_index_censored, brier_score
import numpy as np


def calculate_hybrid_score(
    model,
    X_val,
    y_val,
    horizons=[24, 48, 72],
    weights=[0.3, 0.4, 0.3],
    eps=1e-6
):

    # =====================
    # 1. C-index
    # =====================

    risk_preds = model.predict(X_val)

    c_index = concordance_index_censored(
        y_val["event"],
        y_val["time"],
        risk_preds
    )[0]

    # =====================
    # 2. SAFE horizons (sksurv strict)
    # =====================

    min_time = y_val["time"].min()
    max_time = y_val["time"].max()

    safe_horizons = np.clip(
        horizons,
        min_time + eps,
        max_time - eps
    )

    # =====================
    # 3. Event probabilities
    # =====================

    surv_fns = model.predict_survival_function(X_val)

    event_probs = extract_event_probs(
        surv_fns,
        safe_horizons,
        eps=eps
    )   # shape: (n_samples, len(horizons))

    event_probs_all = extract_event_probs(
        surv_fns,
        [12, 24, 48, 72],
        eps=eps
    )

    # =====================
    # 4. Censor-aware Brier
    # =====================

    brier_vals = []

    for i, t in enumerate(safe_horizons):

        # sksurv expects SURVIVAL prob
        surv_probs = 1.0 - event_probs[:, i]

        _, bs = brier_score(
            y_val,
            y_val,
            surv_probs,
            t
        )

        brier_vals.append(bs[0])

    brier_vals = np.array(brier_vals)

    weighted_brier = np.average(
        brier_vals,
        weights=weights[:len(brier_vals)]
    )

    # =====================
    # 5. Hybrid
    # =====================

    hybrid = 0.3 * c_index + 0.7 * (1.0 - weighted_brier)

    # =====================
    # 6. Return (clean)
    # =====================

    return {
        "hybrid": hybrid,
        "c_index": c_index,
        "weighted_brier": weighted_brier,
        "brier_each": dict(zip(horizons[:len(brier_vals)], brier_vals)),

        # event probabilities aligned with horizons
        "prob_12h": event_probs_all[:, 0],
        "prob_24h": event_probs_all[:, 1],
        "prob_48h": event_probs_all[:, 2],
        "prob_72h": event_probs_all[:, 3],
        "risk_score":risk_preds
    }


In [9]:
# import pandas as pd

# =====================================
# Helper: attach predictions to dataframe
# =====================================
def attach_survival_predictions(base_df, 
                                oof_prob_12h,
                                oof_prob_24h,
                                oof_prob_48h,
                                oof_prob_72h,
                                risk_scores):

    df_preds = pd.DataFrame({
        "prob_12h": oof_prob_12h,
        "prob_24h": oof_prob_24h,
        "prob_48h": oof_prob_48h,
        "prob_72h": oof_prob_72h,
        "risk_score": risk_scores
    }, index=base_df.index)

    return pd.concat([base_df, df_preds], axis=1)


In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sksurv.util import Surv
from sksurv.ensemble import RandomSurvivalForest

from sksurv.ensemble import GradientBoostingSurvivalAnalysis


# ===============================
# Prepare target
# ===============================

time = train["time_to_hit_hours"].values
event = train["event"].astype(bool).values

y = Surv.from_arrays(event, time)


# ===============================
# Stratified CV
# ===============================

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

hybrid_scores = []
cindex_scores = []
brier_scores = []


# ===============================
# OOF containers
# ===============================

oof_event_prob_12h = np.zeros(len(X))
oof_event_prob_24h = np.zeros(len(X))
oof_event_prob_48h = np.zeros(len(X))
oof_event_prob_72h = np.zeros(len(X))

oof_risk = np.zeros(len(X))   # optional


# ===============================
# TEST containers (accumulate)
# ===============================

test_event_prob_12h_sum = np.zeros(len(X_test))
test_event_prob_24h_sum = np.zeros(len(X_test))
test_event_prob_48h_sum = np.zeros(len(X_test))
test_event_prob_72h_sum = np.zeros(len(X_test))

test_risk_sum = np.zeros(len(X_test))


# ===============================
# CV loop
# ===============================

for fold, (train_idx, val_idx) in enumerate(skf.split(X, event), 1):

    print(f"\n===== Fold {fold} =====")

    X_train_df, X_val_df = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # -------------------------
    # Scale
    # -------------------------

    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train_df)
    X_val   = scaler.transform(X_val_df)
    X_test_scaled = scaler.transform(X_test)

    # -------------------------
    # RSF
    # -------------------------

    # rsf = RandomSurvivalForest(
    #     n_estimators=300,
    #     min_samples_split=10,
    #     min_samples_leaf=5,
    #     max_features="sqrt",
    #     n_jobs=-1,
    #     random_state=42
    # )

    # rsf= RandomSurvivalForest(n_estimators=100)

    gbsa = GradientBoostingSurvivalAnalysis(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3
    )


    gbsa.fit(X_train, y_train)

    # -------------------------
    # Risk OOF
    # -------------------------

    oof_risk[val_idx] = gbsa.predict(X_val)

    # -------------------------
    # Hybrid + event probs (VAL)
    # -------------------------

    metrics = calculate_hybrid_score(
        model=gbsa,
        X_val=X_val,
        y_val=y_val,
        horizons=[24, 48, 72],
        weights=[0.3, 0.4, 0.3]
    )

    # -------------------------
    # SAVE OOF EVENT PROBS
    # -------------------------

    oof_event_prob_12h[val_idx] = metrics["prob_12h"]
    oof_event_prob_24h[val_idx] = metrics["prob_24h"]
    oof_event_prob_48h[val_idx] = metrics["prob_48h"]
    oof_event_prob_72h[val_idx] = metrics["prob_72h"]

    oof_risk[val_idx]=metrics["risk_score"]

    # -------------------------
    # TEST predictions (this fold)
    # -------------------------

    test_surv_fns = gbsa.predict_survival_function(X_test_scaled)

    test_event_probs = extract_event_probs(
        test_surv_fns,
        time_points=[12, 24, 48, 72]
    )

    # accumulate
    test_event_prob_12h_sum += test_event_probs[:, 0]
    test_event_prob_24h_sum += test_event_probs[:, 1]
    test_event_prob_48h_sum += test_event_probs[:, 2]
    test_event_prob_72h_sum += test_event_probs[:, 3]

    test_risk_sum += gbsa.predict(X_test_scaled)

    # -------------------------
    # Logs
    # -------------------------

    print("Hybrid :", round(metrics["hybrid"], 4))
    print("C-index:", round(metrics["c_index"], 4))
    print("WBrier :", round(metrics["weighted_brier"], 4))
    print("Brier each:", metrics["brier_each"])

    hybrid_scores.append(metrics["hybrid"])
    cindex_scores.append(metrics["c_index"])
    brier_scores.append(metrics["weighted_brier"])


# ===============================
# Average TEST predictions
# ===============================

n_folds = skf.n_splits

test_event_prob_12h = test_event_prob_12h_sum / n_folds
test_event_prob_24h = test_event_prob_24h_sum / n_folds
test_event_prob_48h = test_event_prob_48h_sum / n_folds
test_event_prob_72h = test_event_prob_72h_sum / n_folds

test_risk = test_risk_sum / n_folds


# ===============================
# Attach to test dataframe
# ===============================

sub["prob_12h"] = test_event_prob_12h
sub["prob_24h"] = test_event_prob_24h
sub["prob_48h"] = test_event_prob_48h
sub["prob_72h"] = test_event_prob_72h
# test["risk_score"]    = test_risk

# ===============================
# Final CV results
# ===============================

print("\n==============================")
print("MEAN Hybrid :", np.mean(hybrid_scores))
print("MEAN C-index:", np.mean(cindex_scores))
print("MEAN WBrier :", np.mean(brier_scores))
print("STD Hybrid  :", np.std(hybrid_scores))



===== Fold 1 =====
Hybrid : 0.983
C-index: 0.9472
WBrier : 0.0016
Brier each: {24: np.float64(0.0014764777233544444), 48: np.float64(0.001292418840488827), 72: np.float64(0.0020438540053043966)}

===== Fold 2 =====
Hybrid : 0.9611
C-index: 0.9327
WBrier : 0.0267
Brier each: {24: np.float64(0.045343109297819276), 48: np.float64(0.02379902425939214), 72: np.float64(0.011895505919655276)}

===== Fold 3 =====
Hybrid : 0.957
C-index: 0.9402
WBrier : 0.0358
Brier each: {24: np.float64(0.05759114458958447), 48: np.float64(0.04423738527731059), 72: np.float64(0.002656398858114082)}

===== Fold 4 =====
Hybrid : 0.9881
C-index: 0.9635
WBrier : 0.0014
Brier each: {24: np.float64(0.0011630704074613067), 48: np.float64(0.0011118560076707265), 72: np.float64(0.001964412459452532)}

===== Fold 5 =====
Hybrid : 0.9631
C-index: 0.9161
WBrier : 0.0168
Brier each: {24: np.float64(0.04032222085553661), 48: np.float64(0.007589803825689087), 72: np.float64(0.005541561815703584)}

MEAN Hybrid : 0.9704723985

In [11]:
# ==============================
# MEAN Hybrid : 0.970465575446066
# MEAN C-index: 0.9399400388519539
# MEAN WBrier : 0.016452051727886035
# STD Hybrid  : 0.01258213600136669

In [12]:
# df=pd.DataFrame()
train_with_oof = attach_survival_predictions(
    base_df=train,
    oof_prob_12h=oof_event_prob_12h,
    oof_prob_24h=oof_event_prob_24h,
    oof_prob_48h=oof_event_prob_48h,
    oof_prob_72h=oof_event_prob_72h,
    risk_scores=oof_risk
)

train_with_oof
# attach_survival_predictions(base_df, event_probs, risk_scores, index)

,event_id,num_perimeters_0_5h,dt_first_last_0_5h,low_temporal_resolution_0_5h,area_first_ha,area_growth_abs_0_5h,area_growth_rel_0_5h,area_growth_rate_ha_per_h,log1p_area_first,log1p_growth,...,event_start_hour,event_start_dayofweek,event_start_month,time_to_hit_hours,event,prob_12h,prob_24h,prob_48h,prob_72h,risk_score
0,10892457,3,4.265188,0,79.696304,2.875935,0.036086,0.674281,4.390693,1.354787,...,19,4,5,18.892512,0,0.020974,0.059842,0.080008,0.094717,-1.309998
1,11757157,2,1.169918,0,8.946749,0.000000,0.000000,0.000000,2.297246,0.000000,...,4,4,6,22.048108,1,0.994820,1.000000,1.000000,1.000000,4.522606
2,11945086,4,4.777526,0,106.482638,0.000000,0.000000,0.000000,4.677329,0.000000,...,22,4,8,0.888895,1,1.000000,1.000000,1.000000,1.000000,5.418506
3,12044083,1,0.000000,1,67.631125,0.000000,0.000000,0.000000,4.228746,0.000000,...,20,5,8,60.953021,0,0.009610,0.030758,0.040361,0.122691,-1.778208
4,12052347,2,4.975273,0,35.632874,0.000000,0.000000,0.000000,3.600946,0.000000,...,21,5,7,44.990274,0,0.009495,0.028468,0.042607,0.143402,-1.662531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,97075632,1,0.000000,1,51.295195,0.000000,0.000000,0.000000,3.956904,0.000000,...,17,6,6,66.340624,0,0.008426,0.027119,0.039836,0.114123,-1.779012
217,97362560,2,1.127102,0,1.176991,0.000000,0.000000,0.000000,0.777943,0.000000,...,18,1,7,5.694898,1,0.448838,0.863299,0.930014,0.999534,2.191168
218,97805715,2,3.710653,0,71.946930,0.000000,0.000000,0.000000,4.289732,0.000000,...,18,1,9,44.011253,0,0.012955,0.038705,0.057775,0.190678,-1.350079
219,99071478,1,0.000000,1,20.223659,0.000000,0.000000,0.000000,3.055117,0.000000,...,15,0,8,22.975783,1,0.400227,0.818704,0.897934,0.998616,2.038156


In [13]:
train_with_oof.set_index("event_id",inplace=True)
train_with_oof.iloc[:,-7:]

,time_to_hit_hours,event,prob_12h,prob_24h,prob_48h,prob_72h,risk_score
event_id,,,,,,,
10892457,18.892512,0,0.020974,0.059842,0.080008,0.094717,-1.309998
11757157,22.048108,1,0.994820,1.000000,1.000000,1.000000,4.522606
11945086,0.888895,1,1.000000,1.000000,1.000000,1.000000,5.418506
12044083,60.953021,0,0.009610,0.030758,0.040361,0.122691,-1.778208
12052347,44.990274,0,0.009495,0.028468,0.042607,0.143402,-1.662531
...,...,...,...,...,...,...,...
97075632,66.340624,0,0.008426,0.027119,0.039836,0.114123,-1.779012
97362560,5.694898,1,0.448838,0.863299,0.930014,0.999534,2.191168
97805715,44.011253,0,0.012955,0.038705,0.057775,0.190678,-1.350079


In [14]:
# import pandas as pd

# # Initialize a list to store the probabilities
# submission_data = []

# for fn in surv_funcs:
#     # fn(t) gives the probability of survival at time t
#     # 1 - fn(t) gives the probability of the event having occurred
#     probs = [1 - fn(t) for t in time_points]
#     submission_data.append(probs)

# # Create the DataFrame
# submission_df = pd.DataFrame(submission_data, columns=['prob_12h', 'prob_24h', 'prob_48h', 'prob_72h'])

# # Add the event_id from your original test set
# submission_df['event_id'] = test['event_id'].values

# # Reorder columns to match the required schema
# submission_df = sub[['event_id', 'prob_12h', 'prob_24h', 'prob_48h', 'prob_72h']]

from datetime import datetime

time = datetime.now().strftime("%Y%m%d_%H%M%S")

sub.to_csv("submission_surv_{}.csv".format(time), index=False)